In [1]:

import numpy 
import matplotlib.pyplot as plt
import pandas 
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import array

from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape,Conv2D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
#from keras.layers.pooling import GlobalAveragePooling1D
#from keras.optimizers import SGD
from keras.utils import np_utils
#from keras.models import Model

import numpy as np
import cv2
from keras.preprocessing.image import img_to_array











import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator


/Users/SindhuPhani/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

time_steps=19
# load the dataset
dataframe = pandas.read_csv('./Trainold.csv')
dataset = dataframe.values

scaler = MinMaxScaler(feature_range=(0, 1))

           
#print(dataset)

# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)
    
    

print(len(per_vid))



trainX=[]
trainY=[]

# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    histValuesList=[]
    scoreList=[]
    # if we have less than 20 datapoints for a vid we skip over the
    # vid assuming something is missing in the raw data
    
    total = vid.iloc[:,4:20].values
    vidImPath=vid.iloc[:,0:2].values    
    
    if len(total) < time_steps :
        continue
    scoreVal=vid["Score"].values[0] + 1    
    #max_total_for_vid = scoreVal.tolist()
    max_total_for_vid = scoreVal.tolist()
    
    for i in range(0,time_steps):
        videoName=vidImPath[i][0]
        imgName=vidImPath[i][1]
        path="./IMAGES/Train/"+videoName+"/"+imgName
        image = cv2.imread(path,0)
        img_arr = img_to_array(image)
        histValuesList.append(img_arr)
        scoreList.append(max_total_for_vid)
        
    trainX.append(histValuesList)
    trainY.append([max_total_for_vid])
    #trainY.append(scoreList)


print(len(trainX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

trainX=numpy.array(trainX)
trainY=numpy.array(trainY)

print(trainX.shape,trainY.shape)


117
19
(114, 19, 250, 700, 1) (114, 1)


In [3]:

time_steps=19
# load the dataset
dataframe = pandas.read_csv('./Test.csv')
dataset = dataframe.values

#print(dataset)

# we group by day so we can process a video at a time.
grouped = dataframe.groupby(dataframe.VidName)

per_vid = []
for _, group in grouped:
    per_vid.append(group)
    
    

print(len(per_vid))



testX=[]
testY=[]

# generate sequences a vid at a time
for i,vid in enumerate(per_vid):
    histValuesList=[]
    scoreList=[]
    # if we have less than 20 datapoints for a vid we skip over the
    # vid assuming something is missing in the raw data
    
    total = vid.iloc[:,4:20].values

    vidImPath=vid.iloc[:,0:2].values

    
    if len(total)<time_steps :
        continue
    scoreVal=vid["Score"].values[0] + 1    
    #max_total_for_vid = scoreVal.tolist()
    max_total_for_vid = scoreVal.tolist()
    
    
    for i in range(0,time_steps):
        #histValuesList.append(total[i])
        #print("Vid and Img name")
        #print(req[i][0],req[i][1])
        videoName=vidImPath[i][0]
        imgName=vidImPath[i][1]
        path="./IMAGES/Test/"+videoName+"/"+imgName
        image = cv2.imread(path,0)
        img_arr = img_to_array(image)
        histValuesList.append(img_arr)
        scoreList.append(max_total_for_vid)

        
    testX.append(histValuesList)
    testY.append([max_total_for_vid])
    #testY.append(scoreList)


print(len(testX[0]))
#trainX = np.array([np.array(xi) for xi in trainX])

testX=numpy.array(testX)
testY=numpy.array(testY)

print(testX.shape,testY.shape)


55
19
(52, 19, 250, 700, 1) (52, 1)


In [4]:
trainX=numpy.array(trainX)
trainX=trainX.reshape(-1,250,700,1)

trainX=trainX/255

trainX=trainX.reshape(-1,19,250,700,1)
print(trainX.shape,trainY.shape)




testX=numpy.array(testX)
testX=testX.reshape(-1,250,700,1)

testX=testX/255
testX=testX.reshape(-1,19,250,700,1)
print(testX.shape,testY.shape)


(114, 19, 250, 700, 1) (114, 1)
(52, 19, 250, 700, 1) (52, 1)


In [5]:
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras import optimizers
from keras import losses
import numpy as np

np.random.seed(3)

timesteps=20; # 20 or frames created
number_of_samples=114; # total no of videos
nb_samples=number_of_samples;
frame_row=250; # image height
frame_col=700; # img width
channels=1; # rgb 

nb_epoch=1;
batch_size=timesteps;

sgd1 = optimizers.SGD(lr=0.005)


nb_epoch=1;
batch_size=timesteps;



model=Sequential();                          

#model.add(TimeDistributed(Convolution2D(32, 3, 3, border_mode='same'), input_shape=trainX.shape[1:]))


model.add(TimeDistributed(Convolution2D(16, 3, 3, input_shape=(250,700,1), activation='relu'),input_shape=trainX.shape[1:]))    
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Convolution2D(32, 3, 3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Convolution2D(64, 3, 3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(256, activation='relu')))
model.add(TimeDistributed(Dropout(.5)))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(TimeDistributed(Dropout(.25)))
model.add(TimeDistributed(Dense(64, activation='relu')))
#model.add(Flatten())
#model.add(TimeDistributed(Dense(1)))
#model.add(Flatten())
model.add(LSTM(20))
model.add(Dense(1))
#model.compile(optimizer=Adam(lr=1e-04), loss='mean_squared_error') 

model.compile(loss='mse', optimizer=sgd1, metrics=['mse'])

model.summary()

/Users/SindhuPhani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), input_shape=(250, 700,..., activation="relu")`
/Users/SindhuPhani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/Users/SindhuPhani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 19, 248, 698, 16)  160       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 19, 124, 349, 16)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 19, 122, 347, 32)  4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 19, 61, 173, 32)   0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 19, 59, 171, 64)   18496     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 19, 29, 85, 64)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 19, 157760)        0         
__________

In [6]:
model.fit(trainX, trainY, batch_size=batch_size, epochs=5, verbose=1)   

Epoch 1/5
114/114 [==============================] - 3898s 34s/step - loss: 2.2347 - mean_squared_error: 2.2347
Epoch 2/5
114/114 [==============================] - 4965s 44s/step - loss: 1.2259 - mean_squared_error: 1.2259
Epoch 3/5
114/114 [==============================] - 6323s 55s/step - loss: 1.0571 - mean_squared_error: 1.0571
Epoch 4/5
114/114 [==============================] - 5391s 47s/step - loss: 1.0367 - mean_squared_error: 1.0367
Epoch 5/5
114/114 [==============================] - 4997s 44s/step - loss: 1.0241 - mean_squared_error: 1.0241


In [7]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

pred=model.predict(testX)

print(pred.shape)
print(testY.shape)



# calculate root mean squared error
#trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
#print('Train Score: %.2f RMSE' % (trainScore))
testScore = mean_squared_error(testY, pred)
print('Test Score: %.2f MSE' % (testScore))

#maeScore = root_mean_squared_error(testY, pred)
#print('RMSE Score: %.2f MAE' % (maeScore))

rmse = np.sqrt(((pred - testY) ** 2).mean(axis=0))
print('RMSE Score: %.2f rmse' % (rmse))

for i in range(0,len(pred)):
    
    print(testY[i],pred[i])

(52, 1)
(52, 1)
Test Score: 1.57 MSE
RMSE Score: 1.25 rmse
[0] [1.7004896]
[0] [1.6688989]
[0] [1.6393205]
[0] [1.7110369]
[1] [1.7345065]
[3] [1.7472938]
[0] [1.7391174]
[4] [1.7410866]
[0] [1.7349585]
[0] [1.7454833]
[3] [1.7415594]
[4] [1.7304785]
[1] [1.6978133]
[2] [1.6363839]
[1] [1.6828238]
[2] [1.6968132]
[1] [1.6957049]
[2] [1.6960124]
[1] [1.7078899]
[0] [1.7344962]
[0] [1.7391795]
[0] [1.724224]
[2] [1.7286818]
[0] [1.6979288]
[0] [1.7262517]
[0] [1.7169774]
[2] [1.7157333]
[1] [1.7038007]
[3] [1.7112952]
[0] [1.7025791]
[0] [1.7115589]
[0] [1.7044338]
[0] [1.7321706]
[2] [1.6998719]
[2] [1.7007699]
[1] [1.6950885]
[1] [1.6839781]
[2] [1.6525481]
[1] [1.66793]
[2] [1.7143855]
[1] [1.698237]
[1] [1.7075626]
[4] [1.7361964]
[1] [1.6878743]
[1] [1.7196648]
[2] [1.6397696]
[0] [1.6517644]
[1] [1.6654024]
[1] [1.6913015]
[1] [1.6910542]
[2] [1.6325033]
[2] [1.702132]


In [8]:
import h5py
model.save(filepath="./CNNLSTM.h5")